In [5]:
import torch
from diffusers import AnimateDiffPipeline, LCMScheduler, MotionAdapter
from diffusers.utils import export_to_video

import numpy as np


In [6]:
def get_video(prompt, resolution):
  image_width, image_height = resolution

  adapter = MotionAdapter.from_pretrained("wangfuyun/AnimateLCM", torch_dtype=torch.float16)

  pipe = AnimateDiffPipeline.from_pretrained("emilianJR/epiCRealism", motion_adapter=adapter, torch_dtype=torch.float16)
  pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config, beta_schedule="linear")
  pipe.load_lora_weights("wangfuyun/AnimateLCM", weight_name="AnimateLCM_sd15_t2v_lora.safetensors", adapter_name="lcm-lora")
  pipe.set_adapters(["lcm-lora"], [0.8])
  pipe.enable_vae_slicing()
  pipe.enable_model_cpu_offload()

  output = pipe(
    prompt=prompt,
    negative_prompt="bad quality, worse quality, low resolution",
    num_frames=30,
    image_width=image_width,
    image_height=image_height,
    guidance_scale=2.0,
    num_inference_steps=24,
    generator=torch.Generator("cpu").manual_seed(0),)
  frames = output.frames[0]  

  return export_to_video(frames, "video.mp4")


In [7]:
def control_center(prompt, platform, types):
    resolution_sizes = {
        "Instagram Reel": (1080, 1920),
        "Instagram Post": (1080, 1080),
        "X Post": (1200, 675),
        "Tik Tok": (1080, 1920)
    }
    platform_resolution = resolution_sizes.get(platform, (1080, 1080))
    types = types.lower()

    if types == "video":
        return get_video(prompt, platform_resolution)
    elif types == "image":
        return get_image(prompt, platform_resolution)
    elif types == "audio":
        return get_voice(prompt, filename="audio.mp3")
    elif types == "only script":
        return get_script(prompt)

In [8]:
control_center("sunrise in the mountains", "Instagram Reel", "video")

100%|██████████| 24/24 [15:12<00:00, 38.02s/it]


'video.mp4'